In [1]:
import pandas as pd
import numpy as np
import numpy as np
import gc # garbage collector
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import lightgbm as lgb
from sklearn.cross_validation import train_test_split

/Users/Rose/anaconda3/envs/fraud/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# read the training data
test_data = pd.read_csv('test.csv')
test_data.sample(5)

,click_id,ip,app,device,os,channel,click_time
3470524,3470526,75007,3,1,20,442,2017-11-10 05:02:14
4287615,4287615,44067,3,1,19,135,2017-11-10 05:18:57
2172645,2172645,20289,27,1,13,122,2017-11-10 04:38:25
15354694,15354694,5314,15,1,13,430,2017-11-10 13:56:55
15520491,15520492,76885,18,1,19,107,2017-11-10 13:59:50


In [3]:
# read the training data
train_data = pd.read_csv('train.csv', nrows=10000000)
train_data.head()


,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0


In [4]:
# check the data column dtype
train_data['click_time'].head()

0    2017-11-06 14:32:21
1    2017-11-06 14:33:34
2    2017-11-06 14:34:12
3    2017-11-06 14:34:52
4    2017-11-06 14:35:08
Name: click_time, dtype: object

The data data type is object, we need to change the column to datetime. We also want to gete the specific day and hour of the click time.

In [5]:
def change_date(df, time):
    r_time = pd.to_datetime(time)
    df['hour'] = pd.to_datetime(r_time).dt.hour.astype('uint8')
    df['day'] = pd.to_datetime(r_time).dt.day.astype('uint8')
    return df

In [6]:
# datetime on test.data
change_date(test_data, test_data.click_time)
test_data.head()

,click_id,ip,app,device,os,channel,click_time,hour,day
0,0,5744,9,1,3,107,2017-11-10 04:00:00,4,10
1,1,119901,9,1,3,466,2017-11-10 04:00:00,4,10
2,2,72287,21,1,19,128,2017-11-10 04:00:00,4,10
3,3,78477,15,1,13,111,2017-11-10 04:00:00,4,10
4,4,123080,12,1,13,328,2017-11-10 04:00:00,4,10


In [7]:
# datetime on train_data
change_date(train_data, train_data.click_time)
train_data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,hour,day
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0,14,6
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0,14,6
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0,14,6
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0,14,6
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0,14,6


###### Analyse the label(is_attributed) by the features

In [8]:
def group(dataset, g, x):
    gp = dataset.groupby(by=g)[['channel']].count().reset_index().rename(index=str, columns={'channel': x})
    dataset = dataset.merge(gp, on=g, how='left')
    return dataset

In [9]:
# ip-click_time_combination
x = 'ip_tcount'
g = ['ip','day','hour']
test_data = group(test_data, g, x )
train_data = group(train_data, g, x)

In [10]:
# ip-app_os combination
x = 'ip_app_os_count'
g = ['ip','app','os']
test_data = group(test_data, g, x )
train_data = group(train_data, g, x)

In [11]:
# ip-device_os combination
x = 'ip_device_os_count'
g = ['ip','device','os']
test_data = group(test_data, g, x )
train_data = group(train_data, g, x)

In [12]:
# ip_is_attributed, device combination
x = 'ip_is_attributed_count'
g = ['ip', 'app', 'device', 'os']
test_data = group(test_data, g, x)
train_data = group(train_data, g, x)

###### using mean and variance

In [13]:
# mean hour
print('grouping by : ip_app_chl_mean_hour')
gp = train_data[['ip','app', 'os', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].mean().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_mean_hour'})
train_data = train_data.merge(gp, on=['ip','app', 'channel'], how='left')
# test 
gp = test_data[['ip','app', 'os', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].mean().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_mean_hour'})
test_data = test_data.merge(gp, on=['ip','app', 'channel'], how='left')

# variance hour
print('grouping by : ip_app_chl_mean_hour')
gp = train_data[['ip','app', 'os', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_var_hour'})
train_data = train_data.merge(gp, on=['ip','app', 'channel'], how='left')
# test 
gp = test_data[['ip','app', 'os', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_var_hour'})
test_data = test_data.merge(gp, on=['ip','app', 'channel'], how='left')

grouping by : ip_app_chl_mean_hour
grouping by : ip_app_chl_mean_hour


In [14]:
# split train_data into training and validation
train_data, validation_data = train_test_split(train_data,  train_size=0.80, random_state=0)
print(train_data.shape)
print(validation_data.shape)


(8000000, 16)
(2000000, 16)


In [15]:
# convert the dataset into LGBM format in this fromat:
# lgb.Dataset(data, label=label, feature_name=['c1', 'c2', 'c3'], categorical_feature=['c3'])
y = 'is_attributed'
x = [ 'app','device','os', 'channel', 'hour', 'day', 
              'ip_tcount', 'ip_app_os_count',
              'ip_device_os_count', 'ip_is_attributed_count']
categorical = ['app', 'device', 'os', 'channel', 'hour', 'day']

l_train = lgb.Dataset(train_data[x].values, 
                    label=train_data[y].values, 
                    feature_name=x,
                    categorical_feature=categorical
                        )

l_validation = lgb.Dataset(validation_data[x].values, 
                    label=validation_data[y].values, 
                    feature_name=x,
                    categorical_feature=categorical
                        )


In [16]:
# LGBM parameters
parameters = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric':'auc',
    'learning_rate': 0.1,
    'num_leaves': 150,
    'max_depth':3,  
    'min_data_in_leaf': 1500,
    'min_child_samples': 100,
    'max_bin': 100,  
    'subsample': 0.9,  
    'subsample_freq': 2,
    'colsample_bytree': 0.9,  
    'min_child_weight': 0,  
    'min_split_gain': 0,
    'reg_alpha': 0, 
    'reg_lambda': 0,
}


In [17]:
train_data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,hour,day,ip_tcount,ip_app_os_count,ip_device_os_count,ip_is_attributed_count,ip_app_channel_mean_hour,ip_app_channel_var_hour
7613831,24008,17,1,13,280,2017-11-06 23:20:06,NaN,0,23,6,410,50,374,45,18.901186,25.113213
6501918,50169,1,1,17,349,2017-11-06 22:43:47,NaN,0,22,6,415,13,156,1,21.500000,0.500000
6201119,111122,18,1,8,134,2017-11-06 22:30:04,NaN,0,22,6,52,9,63,9,18.100000,46.988889
8786253,164899,13,1,10,477,2017-11-06 23:48:59,NaN,0,23,6,8,1,5,1,21.750000,0.916667
699619,37135,15,1,35,3,2017-11-06 16:14:37,NaN,0,16,6,681,55,149,55,18.239203,28.115925


In [18]:
# train the model
model = lgb.train(parameters,
                  l_train,
                  valid_sets=[l_train, l_validation],
                  valid_names=['train','valid'],
                  early_stopping_rounds=100,
                  num_boost_round=300,
                  verbose_eval=10,
                  feval=None,
                  )

/Users/Rose/anaconda3/envs/fraud/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/Rose/anaconda3/envs/fraud/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[10]	train's auc: 0.905774	valid's auc: 0.898768
[20]	train's auc: 0.927944	valid's auc: 0.919397
[30]	train's auc: 0.927813	valid's auc: 0.919264
[40]	train's auc: 0.951076	valid's auc: 0.944882
[50]	train's auc: 0.955811	valid's auc: 0.950811
[60]	train's auc: 0.957605	valid's auc: 0.952411
[70]	train's auc: 0.962418	valid's auc: 0.957162
[80]	train's auc: 0.964948	valid's auc: 0.959491
[90]	train's auc: 0.96719	valid's auc: 0.962031
[100]	train's auc: 0.969093	valid's auc: 0.96331
[110]	train's auc: 0.970244	valid's auc: 0.96403
[120]	train's auc: 0.970853	valid's auc: 0.964474
[130]	train's auc: 0.971587	valid's auc: 0.964576
[140]	train's auc: 0.972032	valid's auc: 0.964913
[150]	train's auc: 0.972526	valid's auc: 0.965468
[160]	train's auc: 0.972843	valid's auc: 0.965698
[170]	train's auc: 0.973265	valid's auc: 0.966025
[180]	train's auc: 0.973587	valid's auc: 0.966274
[190]	train's auc: 0.973882	valid's auc: 0.966251

In [19]:
# predict test data
test_pred = pd.DataFrame()
test_pred['click_id'] = test_data['click_id'].astype('int')
test_pred.head()

test_pred['is_attributed'] = model.predict(test_data[x])
test_pred.to_csv('submission_fraud.csv', index=False)